In [8]:
from datetime import datetime, timedelta
from collections import defaultdict
from calendar import monthrange

def parse_date(date_str):
    return datetime.strptime(date_str, "%Y-%m-%d").date()

def days_in_month(year, month):
    return monthrange(year, month)[1]

def generate_monthly_bill(item_list: list, target_month: str) -> dict:
    year, month = map(int, target_month.split("-"))
    month_start = datetime(year, month, 1).date()
    month_end = datetime(year, month, days_in_month(year, month)).date()
    
    grouped = defaultdict(lambda: {"qty": 0, "amount": 0.0})
    
    for item in item_list:
        try:
            start_date = parse_date(item["start_date"])
            stop_date = parse_date(item["stop_date"])
        except Exception as e:
            continue  # skip invalid entries

        # Get overlap period
        active_start = max(start_date, month_start)
        active_end = min(stop_date, month_end)

        if active_start > active_end:
            continue  # Not active in this month

        # Compute number of active days
        active_days = (active_end - active_start).days + 1
        total_days = days_in_month(year, month)
        prorate_factor = active_days / total_days

        # Normalize types
        rate = float(item["rate"])
        qty = int(item["qty"])
        amount = rate * qty * prorate_factor

        billing_period = f"{active_start} to {active_end}"
        group_key = (item["item_code"], rate, billing_period)

        grouped[group_key]["qty"] += qty
        grouped[group_key]["amount"] += round(amount, 2)
    
    # Prepare output
    line_items = []
    total_revenue = 0.0

    for (item_code, rate, billing_period), values in grouped.items():
        line_items.append({
            "item_code": item_code,
            "rate": rate,
            "qty": values["qty"],
            "amount": round(values["amount"], 2),
            "billing_period": billing_period
        })
        total_revenue += values["amount"]

    return {
        "line_items": line_items,
        "total_revenue": round(total_revenue, 2)
    }
